In [2]:

import math
import pandas
with open("burst_adma_with_cpm_multi_sensors_cls/burst_adma_with_cpm_multi_sensors999_cls_all.csv", 'r') as csvfile:
    df = pandas.read_csv(csvfile)
 # The names of all the columns in the data.

#,cls,timestep,id,x,y,heading,speed,acceleration,label,Unnamed: 8,Unnamed: 9
# distance_columns = ['x', 'y']


print(df.columns.values)

print(df.head)


['timestep' 'cpm_id' 'vehicle_id' 'detected_vehicle_id' 'object_type'
 'detected_object_type' 'cls_detected' 'label_detected' 'x_detected'
 'y_detected' 'speed_detected' 'heading_detected' 'acceleration_detected'
 'is_self' 'sensor_id' 'sensor_type' 'sensor_range' 'weather_conditions'
 'visibility_range' 'detection_confidence']
<bound method NDFrame.head of           timestep            cpm_id vehicle_id detected_vehicle_id  \
0              0.0     CPM_bike0_0.0      bike0                 NaN   
1              0.0     CPM_bike0_0.0      bike0                bus0   
2              0.0     CPM_bike0_0.0      bike0               moto0   
3              0.0     CPM_bike0_0.0      bike0                veh0   
4              0.0     CPM_bike0_0.0      bike0                bus0   
...            ...               ...        ...                 ...   
22736086     999.0  CPM_ped140_999.0     ped140               ped42   
22736087     999.0  CPM_ped140_999.0     ped140               ped97   
2

In [3]:
#print(df.head)
#df.replace(np.nan, '')
df.fillna('0', inplace=True)
#print(df.head)

TEXT_EDGE_INDEX = "edge_index"
TEXT_EDGE_WEIGHT = "edge_weight"
TEXT_X = "x_detected"
TEXT_Y = "y_detected"
TEXT_S_ID = "vehicle_id"
TEXT_HEADING = "heading_detected"
TEXT_SPEED = "speed_detected"
TEXT_ACCELERATION = "acceleration_detected"
TEXT_D_ID = "detected_vehicle_id"
TEXT_CLS = "cls_detected"
TEXT_TS = "timestep"
TEXT_LBL = "label_detected"
TEXT_CONFIDENCE = "detection_confidence"
TEXT_IS_SELF = "is_self"
TEXT_SENSOR = "sensor_type"
TEXT_WEATHER = "weather_conditions"
TEXT_SENSOR_RANGE = "sensor_range"
TEXT_VISIBILITY_RANGE = "visibility_range"
TEXT_D_OBJECT_TYPE = "detected_object_type"

In [4]:
ADDITIONAL_WEIGHT_FEATURES = [TEXT_X,TEXT_Y,TEXT_HEADING,TEXT_SPEED,TEXT_ACCELERATION,TEXT_SENSOR_RANGE,TEXT_VISIBILITY_RANGE] 
# +,TEXT_SENSOR,TEXT_WEATHER 

field_props = {}
for field in ADDITIONAL_WEIGHT_FEATURES:
    x_pros = df[field].describe().transpose()
    field_props[field]={
        "max":x_pros["max"],
        "min":x_pros["min"],
        "null":df[field].isnull().values.any(),
        "nan":df[field].isna().values.any()
    } 
field_props

{'x_detected': {'max': 150.8042257557685,
  'min': -2.232187199358367,
  'null': False,
  'nan': False},
 'y_detected': {'max': -32.36973540004372,
  'min': -44.19131725561458,
  'null': False,
  'nan': False},
 'heading_detected': {'max': 363.6158647253167,
  'min': -3.765504332122038,
  'null': False,
  'nan': False},
 'speed_detected': {'max': 31.24241286079568,
  'min': -11.800640929250104,
  'null': False,
  'nan': False},
 'acceleration_detected': {'max': 9.772124141161148,
  'min': -13.840332112702416,
  'null': False,
  'nan': False},
 'sensor_range': {'max': 80.0, 'min': 0.0, 'null': False, 'nan': False},
 'visibility_range': {'max': 100.0, 'min': 30.0, 'null': False, 'nan': False}}

In [ ]:
import numpy as np 
import json
from tqdm import tqdm
import warnings
ADD_FEATURES_AS_SELF_WEIGHT=True
NEGATIVE_WEIGHT = False

warnings.simplefilter(action='ignore', category=FutureWarning)
unique_node_names = np.unique(df[TEXT_S_ID]).tolist()
node_to_index = {unique_node_names[x]:x  for x in range(0, len(unique_node_names))}
#index_to_node = {x: unique_node_names[x] for x in range(0, len(unique_node_names))}

unique_sensor_types = np.unique(df[TEXT_SENSOR]).tolist()
# print(unique_sensor_types)
sensor_to_index = {unique_sensor_types[x]:x  for x in range(0, len(unique_sensor_types))}
print(json.dumps(sensor_to_index))

unique_weather_types = np.unique(df[TEXT_WEATHER]).tolist()
# print(unique_weather_types)
weather_to_index = {unique_weather_types[x]:x  for x in range(0, len(unique_weather_types))}
print(json.dumps(weather_to_index))
# for num_edges in tqdm([0,1,2,3,4,5,50,100,150,200,10000]):
#edges are knowm
to_export = {
    TEXT_EDGE_INDEX : { },
    TEXT_EDGE_WEIGHT : { },
    "time_periods": 1000,
    "y": [],
    "features": [], # x,y,heading,speed,acceleration,+sensor_type,+sensor_range,+weather_conditions,+visibility_range
    "node_labels" : []
}
to_export["node_labels"] = unique_node_names
cnt = 0
for step, d_step in tqdm(df.groupby([TEXT_TS])):
    step = str(int(step))
    arr_target = np.zeros(len(unique_node_names)).tolist()
    arr_features = np.zeros((len(unique_node_names),len(ADDITIONAL_WEIGHT_FEATURES) + 2)).tolist()
    # +,TEXT_SENSOR,TEXT_WEATHER 
    to_export[TEXT_EDGE_INDEX][step] = []
    to_export[TEXT_EDGE_WEIGHT][step] = []
    to_export[TEXT_Y].append(arr_target)
    to_export["features"].append(arr_features)
    # for cls, d_cls in d_step.groupby([TEXT_CLS]):
    # we have the messages now
    # added = {}
    # cur_edge = 0
    for x, row_x in d_step.iterrows():
        lbl = row_x[TEXT_LBL]
        is_self = row_x[TEXT_IS_SELF]
        from_node_index = node_to_index[row_x[TEXT_S_ID]]
        arr_target[from_node_index]=lbl

        additional_fields = {}
        current_field_index = 0
        for field in ADDITIONAL_WEIGHT_FEATURES:
            x_val = 0 if np.isnan(row_x[field]) else row_x[field]
            if not row_x[TEXT_SENSOR] :
                sensor=0
            else:
                sensor=sensor_to_index[row_x[TEXT_SENSOR]]
            #sensor = 0 if not row_x[TEXT_SENSOR] else sensor_to_index[row_x[TEXT_SENSOR]]
            if not row_x[TEXT_WEATHER] :
                weather=0
            else:
                weather=weather_to_index[row_x[TEXT_WEATHER]]
            #weather = 0 if not row_x[TEXT_WEATHER] else weather_to_index[row_x[TEXT_WEATHER]]
            arr_features[from_node_index][current_field_index]=x_val
            additional_fields[field]=x_val
            current_field_index+=1

        arr_features[from_node_index][current_field_index]=sensor
        additional_fields[TEXT_SENSOR]=sensor

        current_field_index+=1
        arr_features[from_node_index][current_field_index]=weather
        additional_fields[TEXT_WEATHER]=weather
        if is_self == 1:
            # if ADD_FEATURES_AS_SELF_WEIGHT:
            for field in ADDITIONAL_WEIGHT_FEATURES:
                to_export[TEXT_EDGE_INDEX][step].append([from_node_index,from_node_index ])
                to_export[TEXT_EDGE_WEIGHT][step].append(additional_fields[field]) 
        else:
            #for y, row_y in d_cls.iterrows():
            # _key = str(x) + "_" + str(y)
            # _key_rev = str(y) + "_" + str(x)
            # if x!=y and not added.get(_key) and not added.get(_key_rev):
            # from_node_id = node_to_index[row_x[TEXT_ID]]
            to_node_id = node_to_index[row_x[TEXT_D_ID]]
            to_export[TEXT_EDGE_INDEX][step].append([from_node_index,to_node_id ])
            # to_export[TEXT_EDGE_INDEX][step].append([to_node_id, from_node_index])
            # a = np.array((x, y))
            # b = np.array((row_x[TEXT_X], row_x[TEXT_Y]))
            # https://stackoverflow.com/a/1401828/1290868
            # dist = np.linalg.norm(a-b) 
            # dist = -dist if NEGATIVE_WEIGHT else dist 
            # dist = lbl if lbl ==0 else (-dist/lbl if NEGATIVE_WEIGHT else dist*lbl )
            to_export[TEXT_EDGE_WEIGHT][step].append(row_x[TEXT_CONFIDENCE])

            for field in ADDITIONAL_WEIGHT_FEATURES:
                to_export[TEXT_EDGE_INDEX][step].append([from_node_index,from_node_index ])
                to_export[TEXT_EDGE_WEIGHT][step].append(additional_fields[field]) 
            # to_export[TEXT_EDGE_WEIGHT][step].append(row_x[TEXT_CONFIDENCE])
            # added[_key]=True
            # added[_key_rev]=True
            # cur_edge +=1
            # if cur_edge > num_edges:
            #     break
            # if cur_edge > num_edges:
            #   break 
with open(f"data/burst_adma_with_cpm_multi_sensors_cls/burst_adma_with_cpm_multi_sensors999_cls_all_{'_negative' if NEGATIVE_WEIGHT else '_positive'}{'_with_features_as_self_edge' if ADD_FEATURES_AS_SELF_WEIGHT else ''}.json", "w") as outfile:
    json.dump(to_export,outfile)

print(f"CPM-enhanced GNN dataset is saved as burst_adma_with_cpm_multi_sensors999_cls_all_{'_negative' if NEGATIVE_WEIGHT else '_positive'}{'_with_features_as_self_edge' if ADD_FEATURES_AS_SELF_WEIGHT else ''}.json")


{"0": 0, "Camera": 1, "LiDAR": 2, "Radar": 3}
{"clear": 0, "foggy": 1, "rainy": 2, "snowy": 3}


100%|██████████| 1000/1000 [1:03:41<00:00,  3.82s/it]
